In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt
import pandas as pd
import statsmodels.api as sms




In [ ]:
house = pd.read_csv('../data/King_County_House_prices_dataset.csv')

house.sqft_lot15.describe()

In [ ]:
# median 7620 quadratfuß
# wenn in einem viertel q25 von der Grundstücksfläche größer als der Median von den gesamten Grundstücksflächen ist fliegt das viertel raus
new_house = house
big = new_house.groupby('zipcode').sqft_lot15.describe()['25%'].reset_index()
big = big[big['25%'] > 7620]
big_list = big.zipcode.to_list()
big_list

In [ ]:
#Date to datetime series and add month_sold as new column 

new_house['date']=pd.to_datetime(new_house['date'],format='%m/%d/%Y')
new_house['month_sold'] = new_house['date'].dt.month


In [ ]:
# take median zipcodes = 282 houses per zipcode
new_house.groupby('zipcode').count().id.describe()

In [ ]:
# filter the zipcodes with more sold houses and add them to the list zips

top_zip = new_house.groupby('zipcode').count().query('id > 282').reset_index()
zips = top_zip.zipcode.to_list()


In [ ]:
# mindest Anforderung Platz
#110 Quadratmeter = 1184 square foot
# all zipcodes where the count of the sold objects higher than the median


new_house = new_house.query('zipcode in @zips and sqft_living > 1184 and bedrooms >= 4 and zipcode not in @big_list')
new_house

In [ ]:
final = new_house[['price', 'bedrooms', 'sqft_living', 'zipcode', 'month_sold', 'lat', 'long']].copy()
final['price_sqft'] = final.price / final.sqft_living


In [ ]:
# Q25 als grenze für quadratmeterpreis = 177
# anschließend schauen in welchen vierteln der Q25 von allen vierteln gleich Q75 ist und diese rausfiltern weil Ghetto
#df[df['length'] <= 0.4]

ghetto = final.groupby('zipcode').price_sqft.describe()["75%"].reset_index()
ghetto = ghetto[ghetto['75%'] < 177]
ghetto_list = ghetto.zipcode.to_list()
ghetto_list
final = final.query('zipcode not in @ ghetto_list')
final

In [ ]:
sns.pairplot(final)

In [ ]:
plt.scatter(final.month_sold, final.price)

In [ ]:
z = final.groupby('zipcode').mean()
fig = px.bar(final, final.month_sold, final.price)
fig.show()

In [ ]:
fig = px.scatter_mapbox(final, lat=final.lat, lon=final.long, width=1000, height=800)
 
fig.update_geos(fitbounds='locations')
fig.update_layout(title="Social neighborhood")
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.show()


In [ ]:
fig = px.scatter_mapbox(house, lat=house.lat, lon=house.long, width=1000, height=800)
 
fig.update_geos(fitbounds='locations')
fig.update_layout(title="Social neighborhood")
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.show()